## 3. AI Assisted Search - Prompt Engineering

Extract key information from NL query

Prompt engineering:
- [Structured output](https://www.youtube.com/watch?v=kE4BkATIl9c&ab_channel=OpenAI)
- Evaluation

Example: https://github.com/Azure-Samples/azure-openai-entity-extraction

In [1]:
from dotenv import load_dotenv
load_dotenv("../.env")

False

In [2]:
from openai import AzureOpenAI

import os

client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_KEY"),  
  api_version = "2025-01-01-preview",
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

In [5]:
import json

with open("sample_query.json", "r") as f:
    samples = json.load(f)

In [6]:
from pydantic import BaseModel
from typing import List, Any
import json

class QueryResponse(BaseModel):
    date_start: str
    date_end: str
    amount: int
    category: str

In [7]:
def _chat_gpt(messages, model="gpt-4o", temp=0, topp=0.1):
    response = client.beta.chat.completions.parse(
        model=model,
        messages=messages,
        temperature=temp,
        max_tokens=2000,
        top_p=topp,
        response_format=QueryResponse
    )   
    
    return response.choices[0].message.parsed

In [14]:
def compare_values(expected, actual):
    print(expected, "\t|", actual)
    if (expected == actual):
        return True
    return False

def evaluate(prompt):
    passed = 0
    for i, sample in enumerate(samples, 1):
        user_query = sample["query"]
        print(f"{i}:", user_query)

        messages = [{"role": "user", "content": prompt + user_query}]

        structued_response = _chat_gpt(messages)
        print("expected \t| actual")
        if compare_values(sample['expected']['date_start'], structued_response.date_start) and \
            compare_values(sample['expected']['date_end'], structued_response.date_end) and \
            compare_values(sample['expected']['amount'], structued_response.amount) and \
            compare_values(sample['expected']['category'], structued_response.category):
            print("PASS\n")
            passed += 1
        else:
            print("FAIL\n")

    accuracy = passed/len(samples)
    print(f"accuracy: {accuracy*100:.0f} %")


In [ ]:
import datetime

#today = datetime.datetime.today().strftime('%Y-%m-%dT%H:%M:%S')
today = "2025-02-10T18:00:00"

In [ ]:
_user_message_init = f"""You are a helpful assistant.

Today is {today}.
You're task is to extract the key information from the user's search query.

Valid categories are:
- travel: 출장, 해외출장
- tuition: 교육, 영어교육, 외국어 교육
- meal: 식대, 식비
- office: 사무용품, 사무용품 구입
- other: 기타

## Example
Today: 2022-02-01 10:00:00
User query: 지난 달 출장 100만원 이상 비용 품의서를 검색
Date start: 2022-01-01
Date end: 2022-01-31
Amount: 1000000
Category: travel

User query: """

In [16]:
evaluate(_user_message_init)

1: 이번 달 3000만원 이상의 품의서를 찾아줘
expected 	| actual
2025-02-01 	| 2025-02-01
2025-02-28 	| 2025-02-10
FAIL

2: 지난 달 100만원 이상의 영어 교육비 품의서를 찾아줘
expected 	| actual
2025-01-01 	| 2025-01-01
2025-01-31 	| 2025-01-31
1000000 	| 1000000
tuition 	| tuition
PASS

3: 오늘 작성된 품의서를 찾아줘
expected 	| actual
2025-02-10 	| 2025-02-10
2025-02-10 	| 2025-02-10
0 	| 0
 	| other
FAIL

4: 지난 주에 작성된 500만원 이상의 품의서를 찾아줘
expected 	| actual
2025-02-02 	| 2025-02-03
FAIL

accuracy: 25 %


In [ ]:
_user_message = f"""You are a helpful assistant.

Today is {today}.
You're task is to extract the key information from the user's search query.

## Instructions

### Date
1. Week start from sunday to saturday.
2. If date range is not specified, set as today.
3. Today day range of date_start and date_end is the same.

### Amount
1. If amount is not specified, set as 0.

### Category
1. Valid categories are:
    - travel: 출장, 해외출장
    - tuition: 교육, 영어교육, 외국어 교육
    - meal: 식대, 식비
    - office: 사무용품, 사무용품 구입
    - other: 기타
2. If category is not specified, set as empty string.

## Example
Today: 2022-02-01 10:00:00
User query: 지난 달 출장 100만원 이상 비용 품의서를 검색
Date start: 2022-01-01
Date end: 2022-01-31
Amount: 1000000
Category: travel

User query: """

In [18]:
evaluate(_user_message)

1: 이번 달 3000만원 이상의 품의서를 찾아줘
expected 	| actual
2025-02-01 	| 2025-02-01
2025-02-28 	| 2025-02-28
30000000 	| 30000000
 	| 
PASS

2: 지난 달 100만원 이상의 영어 교육비 품의서를 찾아줘
expected 	| actual
2025-01-01 	| 2025-01-01
2025-01-31 	| 2025-01-31
1000000 	| 1000000
tuition 	| tuition
PASS

3: 오늘 작성된 품의서를 찾아줘
expected 	| actual
2025-02-10 	| 2025-02-10
2025-02-10 	| 2025-02-10
0 	| 0
 	| 
PASS

4: 지난 주에 작성된 500만원 이상의 품의서를 찾아줘
expected 	| actual
2025-02-02 	| 2025-02-02
2025-02-08 	| 2025-02-08
5000000 	| 5000000
 	| 
PASS

accuracy: 100 %
